# Chapter 7 - Advanced Text Generation Techniques and Tools

## Loading an LLM

In [ ]:
from langchain import LlamaCpp

llm =  LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=512,
    n_ctx=2048,
    seed=42,
    verbose=False,
)

In [ ]:
llm.invoke("Hi! My name is Nhan. What is 1 + 1?")

### Chains

In [2]:
from langchain import PromptTemplate

template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>
"""
prompt = PromptTemplate(
    input_variables=["input_prompt"],
    template=template,
)

In [ ]:
basic_chain = prompt | llm
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Nhan. What is 1 + 1?",
    }
)

### Multiple Chains

In [ ]:
from langchain import LLMChain

template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>
"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

In [ ]:
title.invoke({"summary": "a girl that lost her mother"})

In [ ]:
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>
"""
character_prompt = PromptTemplate(template=template, input_variables=["summary", "title"])
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [ ]:
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main character is: {character}. Only return the story and it can't be longer than one paragraph<|end|>
<|assistant|>
"""
story_prompt = PromptTemplate(template=template, input_variables=["summary", "title", "character"])
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [ ]:
llm_chain = title | character | story

In [ ]:
llm_chain.invoke("a girl that lost her mother")

## Memory

In [ ]:
basic_chain.invoke({"input_prompt": "Hi! My name is Nhan. What is 1 + 1?"})

In [ ]:
basic_chain.invoke({"input_prompt": "What is my name?"})

### ConversationBuffer

In [ ]:
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [ ]:
llm_chain.invoke({"input_prompt": "Hi! My name is Nhan. What is 1+1?"})

In [ ]:
llm_chain.invoke({"input_prompt": "What is my name?"})

### ConversationBufferMemoryWindow

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [ ]:
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten and I am 22 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is 3 + 3?"}) 

In [ ]:
llm_chain.invoke({"input_prompt": "What is my name?"})

In [ ]:
llm_chain.invoke({"input_prompt": "What is my age?"})

### ConversationSummary 

In [ ]:
summary_prompt_template = """<s><|user|>Summarize the conversation and update with the new lines.

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>
"""

summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [ ]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [ ]:
memory.load_memory_variables({})

## Agents

In [4]:
import os
from langchain_openai import ChatOpenAI

openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [5]:
react_template = """Answer the following questions as best you can. You have to access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Though: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["input", "tools", "tool_names", "agent_scratchpad"],
)

In [6]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general quesries.",
    func=search.run
)

tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [7]:
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [8]:
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
I should use a web search engine to find the current price of a MacBook Pro in USD and then use a calculator to convert it to EUR.
Action: duckduck
Action Input: "current price of MacBook Pro in USD"

d:\Anaconda\envs\LLMs-env\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


[snippet: The base model of the M3 Pro starts at $2,499 USD (although exclusive discounts are available in this guide) and is an ideal option for users who want a larger screen and advanced capabilities. To get the best deal on this or any other MacBook Pro, check out AppleInsider's list of the top MacBook Pro deals., title: MacBook Pro 16-inch Best Price M3 Max, 16C CPU, 40C GPU, 48GB, 1TB ..., link: https://prices.appleinsider.com/product/macbook-pro-16-inch-m3/MUW63LL/A], [snippet: For a limited time, you can get the M4 MacBook Pro (Silver) for $1,373 from go-to Apple retailer, Amazon. That's $226 below the M4 MB Pro's $1,599 list price and just $1 shy of its all-time low., title: Apple's excellent M4 MacBook Pro hits all-time low price - Laptop Mag, link: https://www.laptopmag.com/deals/laptops/macbooks/the-5-star-rated-m4-macbook-pro-dips-to-usd1-373-at-amazon], [snippet: With power, performance, and premium style, the MacBook Pro represents the pinnacle of Apple's laptop offering

d:\Anaconda\envs\LLMs-env\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


[snippet: Apple launched its 16-inch M3 MacBook Pro in late 2023, alongside the smaller 14-inch model. This new MacBook Pro lineup is powered by M3 Pro and M3 Max chipsets, both of which offer significant performance enhancements compared to the M1 and M2 MacBook Pros. The 16-inch M3 MacBook Pro is available in Space Black or Silver., title: MacBook Pro 16-inch Best Price M3 Max, 16C CPU, 40C GPU, 48GB, 1TB ..., link: https://prices.appleinsider.com/product/macbook-pro-16-inch-m3/MUW63LL/A], [snippet: Disclosure: If you buy through our links, we may earn commission as an affiliate & Amazon Associate. Apple's MacBook Pro 16-inch with M3 Pro and M3 Max is its largest and most powerful MacBook available. It starts at $2,499 with the M3 Pro processor, 18GB of RAM, and 512GB of storage., title: M3 MacBook Pro 16-Inch 2023 Best Sale Price Deals - AppleInsider, link: https://prices.appleinsider.com/macbook-pro-16-inch-m3], [snippet: Best Buy has clearance 16″ M3 Max Apple MacBook Pros on sale

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The current price of a MacBook Pro in USD is $2,499 and it would cost 2124.15 EUR with an exchange rate of 0.85 EUR for 1 USD.'}